In [1]:
from __future__ import division, print_function
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import pylab as plt
import datetime as dt
from datetime import datetime as ddt, date, time
import math
import statsmodels.api as sm

import seaborn as sns
%pylab inline

import os
from tqdm import  tqdm #tnrange, tqdm_notebook as

from numba import jit
from multiprocessing.dummy import Pool as ThreadPool

Populating the interactive namespace from numpy and matplotlib


In [2]:
@jit#(nopython=True)
def loadMappings(path):
    xl = pd.ExcelFile(path)
    #xl
    map_podp = xl.parse('Подписант КЗ')
    map_zb = xl.parse('Зак_Блок')
    map_dep = xl.parse('Департамент')
    map_upr = xl.parse('Управление')
    map_otd = xl.parse('Отдел')
    map_podp.dropna(inplace = True)
    map_zb.dropna(inplace = True)
    map_dep.dropna(inplace = True)
    map_upr.dropna(inplace = True)
    map_otd.dropna(inplace = True)
    print('Меппинги загружены')
    return map_podp, map_zb, map_dep, map_upr, map_otd
     
#@jit#(nopython=True)
def loadOrderFile(path, page):
    global map_podp, map_zb, map_dep, map_upr,map_otd
    
    xl = pd.ExcelFile(path)
    df = xl.parse(page)
    colsToDrop = ['Год', 'Месяц', 'Номер строки', 'ID для консолидации','ID АС ЦУП мастер-заказа',
              'Комментарий УСПП','Название заказа (КЗ)','ООУ (открытие)','Дата распоряжения о внедрении по КЗ', 'Номер СП',
              'Актуальный номер спецификации','Подписанты', 'Аккаунт менеджер заказа', 'Заказчик',
              'Тип заказа','Название заказа','Менеджера заказа','Задача в заказе (что делали)', 'Табельный номер',
              'Основание командировки (работы в выходные дни)', 'Период командировки (работы в выходные дни)', 
              'Место командировки', 'Запланированный результат', 'Фактический результат','Комментарий МЗ к отчету',
              'Пояснения к изменению объема трудозатрат по результатам согласования с заказчиком', 'e-mail',
              'Согласовано АКТ', 'Отработано по табелю включая командировки, дни',
              'Нормированные по сотруднику утвержденные трудозатраты, дни','Отработано по табелю включая командировки, часы',
              'Фактические трудозатраты введенные сотрудником, часы','Фактические трудозатраты, подтвержжденные менеджером заказа, часы',
              'Нормированный  % загрузки по сотруднику','Нормирование по сотруднику, часы','Стоимость трудозатрат(руб.)',
              'Стоимость трудозатрат, согласованных заказчиком (руб)','Стоимость командировок (руб.)',
              'Стоимость нормированных командировок (руб.)','Стоимость командировок, согласованных заказчиком (руб.)',
              'Стоимость работ в выходные (руб.)','Стоимость работ в выходные, согласованная с заказчиком (руб.)',
              'ИТОГО стоимость работ, согласованная заказчиком(руб.)','Ставка \nСБТ-Банк (руб.)','Этап СП', 
              'Дата начала этапа СП','Базовая дата окончания этапа СП', 'Прогнозная дата окончания этапа СП',
              'Фактическая дата окончания этапа СП', 'Статус этапа СП','Базовая дата окончания уточненного этапа СП','Статус',
              'Дата закрытия КЗ','Дата окончания КЗ','Компания','Дата закрытия на ООУ']
            
    try:
        df.drop(colsToDrop,inplace=True, axis =1)
    except ValueError:
        colsToDrop.remove('Отработано по табелю включая командировки, дни')
        colsToDrop.remove('Отработано по табелю включая командировки, часы')
        colsToDrop = colsToDrop + ['Отработанное по табелю время работы на Sbergile - команду включая командировки, дни',
                                  'Отработано по табелю время работы на Sbergile - команду включая командировки, часы',
                                  'Отработанное по табелю время работы не на Sbergile - команду включая командировки, часы',
                                  'Отработанное по табелю время работы не на Sbergile - команду включая командировки, дни']
        df.drop(colsToDrop,inplace=True, axis =1)
        
    df['Подписант КЗ'].fillna('#nd',inplace=True)
    df['Заказчик КЗ'].fillna('#nd.#nd.#nd.#nd.#nd.#nd',inplace=True)
    df['Заказчик КЗ']=df['Заказчик КЗ']+'.#nd.#nd.#nd.#nd.#nd.#nd'
   
    df['Дата открытия на ООУ'].fillna(method ='ffill',inplace=True)
    df['Статус проекта в портфеле Банка для КЗ'].fillna('#nd',inplace=True)
    df['Уточненный тип договора '].fillna('Fix price',inplace=True)
    df['Уточненный тип СП'].fillna('Разработка',inplace=True)
    df['Уточненный номер этапа СП'].fillna(0,inplace=True)
    #df['Дата начала этапа'].fillna(0,inplace=True)
    #df['Дата окончания этапа'].fillna(0,inplace=True)
    df['Статус уточненного этапа СП'].fillna('#nd',inplace=True)
    df['Статус акта по этапу'].fillna('#nd',inplace=True)
    df['ID АС ЦУП'].fillna('#nd',inplace=True)
    df['Департамент'].fillna('#nd',inplace=True)
    df['Управление'].fillna('#nd',inplace=True)
    df['Отдел'].fillna('#nd',inplace=True)
    
    df['Согласование аккаунт-менеджера'].fillna('#nd',inplace=True)
    df['Функциональный блок'].fillna('#nd',inplace=True)
    try: 
        df['Тип командировки (Зарубежная /По России)'].fillna('#nd',inplace=True)
        df['Работа в Sbergile'].fillna('#nd',inplace=True)
    except KeyError:
        df['Тип командировки (Зарубежная /По России)'] = '#nd'
        df['Работа в Sbergile'] = '#nd'
    df.dropna(inplace = True)
    #Дополнить кодировку блоков
    df['Зак_Блок']  = df['Функциональный блок']
    df.drop(['Функциональный блок'],inplace=True, axis =1)
    #df['Зак_Блок']=df.apply(lambda r: r['Заказчик КЗ'].split('.')[3], axis=1)
    df['Зак_Блок']=df.apply(lambda r: r['Заказчик КЗ'].split('.')[3] if ((r['Зак_Блок'])=='#nd') else r['Зак_Блок'], axis=1)
    df['Зак_стр_п']=df.apply(lambda r: r['Заказчик КЗ'].split('.')[4], axis=1)
    df['Зак_стр_п'].fillna('#nd',inplace=True)
    df['Зак_менеджер']=df.apply(lambda r: r['Заказчик КЗ'].split('.')[5], axis=1)
 
    df.drop('Заказчик КЗ',inplace=True, axis =1)
    
    for index, row in map_podp.iterrows():
        df.loc[df['Подписант КЗ']==row['From'],'Подписант КЗ']=row['To']
    for index, row in map_zb.iterrows():
        df.loc[df['Зак_Блок']==row['From'],'Зак_Блок']=row['To']
    for index, row in map_dep.iterrows():
        df.loc[df['Департамент']==row['From'],'Департамент']=row['To']
    for index, row in map_upr.iterrows():
        df.loc[df['Управление']==row['From'],'Управление']=row['To']
    
    df.loc[df['Категория']=='Billable staff','Категория']='B'
    df.loc[df['Категория']=='Non-billible staff','Категория']='N'
    df.loc[df['Наличие СП']=='#Н/Д','Наличие СП']='Наличие СП'

    df['Грейд']=df['Грейд'].astype('int32')
    print(path)
    return df

#@jit
def loadWorkFile1(path, page):
    global map_podp, map_zb, map_dep, map_upr,map_otd
    xl = pd.ExcelFile(path)
    df = xl.parse(page)
    df = df.iloc[1:]#отбрасываем строчку с номерами
    colsToDrop = ['ID АС ЦУП','Название заказа','Компания','Парковка','Внешний грейд', 'Подписант','Тип заказа',
               'Чел/часы\nпо дням','Плановый процент загрузки\nпо дням','Дата начала списания',  'Дата последнего списания',
               'Чел/дни.1','Фактическая стоимость (руб)','Резерв (п.14-п.17)','Коэффициент использования ресурса, % (п.17/п.14)']
    try:
        df.drop(colsToDrop,inplace=True, axis =1)
    except ValueError:
        colsToDrop.remove('Подписант')
        
        try:
            df.drop(colsToDrop,inplace=True, axis =1)
        except ValueError:
            colsToDrop.remove('Тип заказа')
            df.drop(colsToDrop,inplace=True, axis =1)
    df.dropna(inplace = True)
    
    for index, row in map_dep.iterrows():
        df.loc[df['Департамент']==row['From'],'Департамент']=row['To']
    for index, row in map_upr.iterrows():
        df.loc[df['Управление']==row['From'],'Управление']=row['To']
    
    df.loc[df['Категория ресурса']=='Billable staff','Категория ресурса']='B'
    df.loc[df['Категория ресурса']=='Non-billible staff','Категория ресурса']='N'
    df['Внутренний грейд'].fillna('Внутренний грейд',inplace=True)
    df['Внутренний грейд']=df['Внутренний грейд'].astype('int32')

    print(path)
    return df


#загрузка "41" отчётов
#@jit#(nopython=True)
def loadOrders(path):
    files = os.listdir(path) 
    xlsx = [file for file in files if file.endswith('.xlsx')]
    datas = [loadOrderFile(path+xlsx[i], 'actworkcostreport.rdl') for i in range(len(xlsx))]
    return pd.concat(datas,ignore_index=True)

#загрузка отчётов "БП2"
#@jit#(nopython=True)
def loadWorks1(path):
    files = os.listdir(path) 
    xlsx = [file for file in files if file.endswith('.xlsx')]
    datas = [loadWorkFile1(path+xlsx[i], 'planningresources1.rdl') for i in range(len(xlsx))]
    return pd.concat(datas, ignore_index=True)

In [3]:
#грузим данные 
map_podp, map_zb, map_dep, map_upr,map_otd = loadMappings('data\\mapping\\mapping.xlsx')
pre_df = loadOrders('data\\41\\')
#pre_df.drop_duplicates(inplace=True)
pre_df2 = loadWorks1('data\\БП2\\')
#pre_df2.drop_duplicates(inplace=True)


Меппинги загружены
data\41\41 2015 H1.xlsx
data\41\41 2015 H2.xlsx
data\41\41 2016 All.xlsx
data\41\41_2017H.xlsx
data\41\41_2017Q3.xlsx
data\БП2\БП2 (57) 10.xlsx
data\БП2\БП2 (57) 11.xlsx
data\БП2\БП2 (57) 12.xlsx
data\БП2\БП2 (57) 15-1.xlsx
data\БП2\БП2 (57) 15-10.xlsx
data\БП2\БП2 (57) 15-11.xlsx
data\БП2\БП2 (57) 15-12.xlsx
data\БП2\БП2 (57) 15-2.xlsx
data\БП2\БП2 (57) 15-3.xlsx
data\БП2\БП2 (57) 15-4.xlsx
data\БП2\БП2 (57) 15-5.xlsx
data\БП2\БП2 (57) 15-6 версия.xlsx
data\БП2\БП2 (57) 15-7.xlsx
data\БП2\БП2 (57) 15-8.xlsx
data\БП2\БП2 (57) 15-9.xlsx
data\БП2\БП2 (57) 1_c.xlsx
data\БП2\БП2 (57) 2.xlsx
data\БП2\БП2 (57) 3.xlsx
data\БП2\БП2 (57) 4.xlsx
data\БП2\БП2 (57) 5.xlsx
data\БП2\БП2 (57) 6.xlsx
data\БП2\БП2 (57) 7.xlsx
data\БП2\БП2 (57) 8.xlsx
data\БП2\БП2 (57) 9.xlsx
data\БП2\БП2-2017 второе полугодие.xlsx
data\БП2\БП2-2017 первое полугодие.xlsx


In [6]:
#закомментировать если данные не менялись
#pre_df2.to_excel('data/pre_df2_1.xlsx')
#pre_df.to_excel('data/pre_df_1.xlsx')

#раскомментировать если работаем с тем, что раньше грузили
#xl = pd.ExcelFile('data/pre_df2.xlsx')
#pre_df2 = xl.parse('Sheet1')
#xl = pd.ExcelFile('data/pre_df.xlsx')
#pre_df2 = xl.parse('Sheet1')
!telegram-send 'файлы_загружены'

In [3]:
#в качестве менеджера клиентского заказа, аккаунт менеджера и менеджера заказчика указываем того у которого больше всего записей по этому КЗ
#@jit(nopython=True)
@jit
def fillMax( fFill, fMaxCount):
    global pre_df
    df_mgr_count = pd.DataFrame(pre_df.groupby(['ID АРИАДНЫ',fFill])[fMaxCount].count()).reset_index()
    df_mgr_count = df_mgr_count.rename(columns={fMaxCount: 'кол-во'})
    df_mgr_count = df_mgr_count.groupby(['ID АРИАДНЫ'], as_index=False).max()
    #df_mgr_count.drop(['кол-во'], inplace=True, axis=1) 
    for index, row in df_mgr_count.iterrows():
        pre_df.loc[pre_df['ID АРИАДНЫ']==row['ID АРИАДНЫ'],fFill]=row[fFill]

#в случае, если по заказу есть данные и #nd -пишем нужные нам данные 
@jit#(nopython=True)
def fillND( fFill, fMaxCount):
    global pre_df
    df_mgr_count = pd.DataFrame(pre_df.groupby(['ID АРИАДНЫ',fFill])[fMaxCount].count()).reset_index()
    df_mgr_count = df_mgr_count.rename(columns={fMaxCount: 'кол-во'})
    df_mgr_count = df_mgr_count[df_mgr_count[fFill]!='#nd']
    df_mgr_count = df_mgr_count.groupby(['ID АРИАДНЫ'], as_index=False).max()
    for index, row in df_mgr_count.iterrows():
        pre_df.loc[pre_df['ID АРИАДНЫ']==row['ID АРИАДНЫ'],fFill]=row[fFill]

#делаем ключ уникальным pre_df
@jit#(nopython=True)
def makeKeyUniqe1():
    global pre_df
    
    df_id_ar = pd.DataFrame(pre_df.groupby(['key1'])['Подписант КЗ'].count()).reset_index()
    df_id_ar.drop(['Подписант КЗ'], inplace=True, axis=1) 
    for index, row in df_id_ar.iterrows():
        df_loc = pre_df[pre_df['key1']==row['key1']][['key1','ч/д_факт']]
        df_loc['ind']=df_loc.index
        df_loc.sort_values(['ч/д_факт'], ascending=[False], inplace=True)
        df_loc_sort = pd.DataFrame(df_loc[['key1','ч/д_факт','ind']]).reset_index()
        #print(df_loc_sort.head(2))
        df_loc_sort['row_n']=df_loc_sort.index
        #df_loc_sort.drop(['index'], inplace=True, axis=1) 
        df_loc_sort['row_n']=df_loc_sort['key1']+'_'+df_loc_sort['row_n'].astype(str)
        for index, row in df_loc_sort.iterrows():
            pre_df.loc[(pre_df.index==row['ind']),'key2']=row['row_n']

#делаем ключ уникальным pre_df2
@jit#(nopython=True)
def makeKeyUniqe2():
    global pre_df2
    
    df_id_ar = pd.DataFrame(pre_df2.groupby(['key1'])['Категория'].count()).reset_index()
    df_id_ar.drop(['Категория'], inplace=True, axis=1) 
    for index, row in df_id_ar.iterrows():
        df_loc = pre_df2[pre_df2['key1']==row['key1']][['key1','ч/д_план']]
        df_loc['ind']=df_loc.index
        df_loc.sort_values(['ч/д_план'], ascending=[False], inplace=True)
        df_loc_sort = pd.DataFrame(df_loc[['key1','ч/д_план','ind']]).reset_index()
        df_loc_sort['row_n']=df_loc_sort.index
        #df_loc_sort.drop(['index'], inplace=True, axis=1) 
        df_loc_sort['row_n']=df_loc_sort['key1']+'_'+df_loc_sort['row_n'].astype(str)
        for index, row in df_loc_sort.iterrows():
            pre_df2.loc[(pre_df2.index==row['ind']),'key2']=row['row_n']

@jit#(nopython=True)
def makeFullDS(): #заполнение пропусков наиболее часто встречающимся значением по заказу
    global df

    cols_id = ['ID АРИАДНЫ']
    #'ID АС ЦУП',
    colsToFill = ['Аккаунт менеджер КЗ','Дата открытия на ООУ',
                  'Зак_Блок','Зак_менеджер','Зак_стр_п','Менеджер (КЗ)','Наличие СП',
                  'Подписант КЗ','Прогноз/Факт','Работа в Sbergile',
                  'Согласование аккаунт-менеджера','Справочник типа заказа',
                  'Статус акта по этапу','Статус проекта в портфеле Банка для КЗ',
                  'Тип КЗ', 'Тип СП', 'Тип договора', 'Тип затрат',
                  'Тип командировки (Зарубежная /По России)',
                  'Уточненный номер этапа СП', 'Уточненный тип СП',
                  'Уточненный тип договора ']
    groupCols = cols_id + colsToFill
    df_orders = pd.DataFrame(df.groupby(groupCols)['key2'].count()).reset_index()
    df_orders = df_orders.rename(columns={'key2': 'кол-во'})
    df_orders = df_orders.groupby('ID АРИАДНЫ', as_index=False).max()
    
    for index, row in df_orders.iterrows():
        for field in colsToFill: 
            df.loc[df['ID АРИАДНЫ']==row['ID АРИАДНЫ'],field]=row[field]


In [8]:
pbar_link = tqdm(total=7)
#приведение колонок к нужному типу данных
pre_df['Дата открытия на ООУ'] = pd.to_datetime(pre_df['Дата открытия на ООУ'])
pre_df['Дата начала этапа'] = pd.to_datetime(pre_df['Дата начала этапа'])
pre_df['Дата окончания этапа'] = pd.to_datetime(pre_df['Дата окончания этапа'])
pre_df2['Дата начала'] = pd.to_datetime(pre_df2['Дата начала'])
pre_df2['Дата окончания'] = pd.to_datetime(pre_df2['Дата окончания'])

pre_df2['Дл раб план'] = pre_df2['Дата окончания'].map(dt.datetime.toordinal) - pre_df2['Дата начала'].map(dt.datetime.toordinal)
pre_df2['Дата начала'].fillna(method = 'ffill',inplace=True)
pre_df['Дата начала этапа'].fillna(method = 'ffill',inplace=True)
pre_df['Дл раб факт'] = pre_df['Дата окончания этапа'].map(dt.datetime.toordinal) - pre_df['Дата начала этапа'].map(dt.datetime.toordinal)

pre_df = pre_df.rename(columns={'Трудозатраты,  подтвержденные заказчиком, дни': 'ч/д_факт'})
pre_df2 = pre_df2.rename(columns={'Чел/дни': 'ч/д_план','ID заказа Ариадны':'ID АРИАДНЫ','Внутренний грейд':'Грейд','Категория ресурса':'Категория','ФИО/Наименование':'ФИО'})

#устраняем противоречия в заказчиках, и подписантах
fillMax('Менеджер (КЗ)','Подписант КЗ')
pre_df['Менеджер (КЗ)'].fillna('#nd',inplace=True)
pbar_link.update(1)
fillMax('Аккаунт менеджер КЗ','Подписант КЗ')
pre_df['Аккаунт менеджер КЗ'].fillna('#nd',inplace=True)
pbar_link.update(1)
fillMax('Зак_менеджер','Подписант КЗ')
df['Зак_менеджер'].fillna('#nd',inplace=True)
pbar_link.update(1)
fillND('Зак_Блок','Подписант КЗ')
pre_df['Зак_Блок'].fillna('#nd',inplace=True)
pbar_link.update(1)
fillND('Согласование аккаунт-менеджера','Подписант КЗ')
pbar_link.update(1)
#формируем ключи для склеивания
pre_df['key1'] = pre_df['ID АРИАДНЫ'].astype(str)+'_'+ pre_df['ФИО'].astype(str)#+ pre_df['Управление'].astype(str)+'_'+ pre_df['Должность'].astype(str)+'_'+ pre_df['Грейд'].astype(str)+'_'+ pre_df['ФИО'].astype(str)
pre_df2['key1'] = pre_df2['ID АРИАДНЫ'].astype(str)+'_'+ pre_df2['ФИО'].astype(str)#+'_'+ pre_df2['Управление'].astype(str)+'_'+ pre_df2['Должность'].astype(str)+'_'+ pre_df2['Грейд'].astype(str)+'_'+ pre_df2['ФИО'].astype(str)
makeKeyUniqe1()
pbar_link.update(1)
makeKeyUniqe2()

pre_df.drop(['key1'], inplace=True, axis=1)
pre_df2.drop(['key1'], inplace=True, axis=1)

pbar_link.update(1)
pbar_link.close()

100%|████████████████████████████████████████| 7/7 [5:32:33<00:00, 4758.87s/it]


In [9]:
pre_df2.to_excel('data/pre_df_u2_1.xlsx')
pre_df.to_excel('data/pre_df_u_1.xlsx')


#раскомментировать если работаем с тем, что раньше грузили
#xl = pd.ExcelFile('data/pre_df_u2.xlsx')
#pre_df2 = xl.parse('Sheet1')
#xl = pd.ExcelFile('data/pre_df_u.xlsx')
#pre_df = xl.parse('Sheet1')

!telegram-send 'связи_построены'

In [10]:
#объединяем плановые и фактические значения 
pbar_df = tqdm(total=12)

df = pre_df.merge(pre_df2, how ='outer', on=('key2'),suffixes=('', '_y'))
pbar_df.update(1)
#дополняем факт известными плановыми значениями
df['ID АРИАДНЫ'] = df.apply(lambda r: r['ID АРИАДНЫ_y'] if (pd.isnull(r['ID АРИАДНЫ'])) else r['ID АРИАДНЫ'], axis=1)
pbar_df.update(1)
df['Грейд'] = df.apply(lambda r: r['Грейд_y'] if (pd.isnull(r['Грейд'])) else r['Грейд'], axis=1)
pbar_df.update(1)
df['Департамент'] = df.apply(lambda r: r['Департамент_y'] if (pd.isnull(r['Департамент'])) else r['Департамент'], axis=1)
pbar_df.update(1)
df['Управление'] = df.apply(lambda r: r['Управление_y'] if (pd.isnull(r['Управление'])) else r['Управление'], axis=1)
pbar_df.update(1)
df['Должность'] = df.apply(lambda r: r['Должность_y'] if (pd.isnull(r['Должность'])) else r['Должность'], axis=1)
pbar_df.update(1)
df['ФИО'] = df.apply(lambda r: r['ФИО_y'] if (pd.isnull(r['ФИО'])) else r['ФИО'], axis=1)
pbar_df.update(1)
df['Категория'] = df.apply(lambda r: r['Категория_y'] if (pd.isnull(r['Категория'])) else r['Категория'], axis=1)
pbar_df.update(1)
df['Статус уточненного этапа СП'] = df.apply(lambda r: 'В ожидании' if (pd.isnull(r['Категория'])) else r['Категория'], axis=1)
pbar_df.update(1)
#подчищаем задвоенные колонки
colsToDrop = ['ID АРИАДНЫ_y','Департамент_y','Управление_y','Должность_y','Грейд_y','ФИО_y','Категория_y']
df.drop(colsToDrop,inplace=True, axis =1)
pbar_df.update(1)

makeFullDS()
pbar_df.update(1)

#заводим поле по которому отличаем то, что есть, от того, что надо прогнозировать
df['Статус']=df.apply(lambda r: 1 if (r['Итого стоимость работ (руб.)']>=0) else 0, axis=1)
pbar_df.update(1)

pbar_df.close()    

100%|███████████████████████████████████████| 15/15 [1:50:26<00:00, 441.78s/it]


In [4]:
#df.tail(5)

In [12]:
#записываем промежуточный результат, чтобы потом избежать предрасчёта ключей
df.to_excel('data/df_g.xlsx')

#сверки того, что всё загрузилось и склеилось корректно
print(pre_df['Итого стоимость работ (руб.)'].sum())
print(df['Итого стоимость работ (руб.)'].sum())
print(pre_df['ч/д_факт'].sum())
print(df['ч/д_факт'].sum())
print(pre_df['Дл раб факт'].sum())
print(df['Дл раб факт'].sum())

print(pre_df2['Стоимость (руб)'].sum())
print(df['Стоимость (руб)'].sum())
print(pre_df2['Дл раб план'].sum())
print(df['Дл раб план'].sum())
print(pre_df2['ч/д_план'].sum())
print(df['ч/д_план'].sum())

!telegram-send 'таблицы_нормализованы'

42424761649.56242
42424761649.56242
2896666.35500013
2896666.35500013
101290097
101290097.0
51443520379.7049
51443520379.70531
79990748
79990748.0
3412339.9581012353
3412339.958101235


In [11]:
#сохраняем, чтобы потом не париться, пропустить - если не надо сохранять 
#df.to_csv('data\\df.csv',index_label = 'ind')
